<font size="4">**Детализированная картина. Расчет плотности финансовых организаций (ФО) по регионам РФ и построение тепловой карты с учетом метрик плотности:**</font>

<font size="4">- количество организаций на квадратный километр (density_per_km2);</font>

<font size="4">- количество организаций на 100 тысяч населения (density_per_100k).</font>

<font size="4">**Часть 1. Подготовка таблицы рейтингов ФО с учетом метрик плотности.**</font>

In [2]:
import pandas as pd
# import folium
# from folium.plugins import HeatMap, MarkerCluster

In [3]:
# Загрузка данных 
df = pd.read_excel('Свод ГО и подразделения.xlsx') 
regions_df = pd.read_excel('russia_data.xlsx')

In [4]:
# Предобработка данных
df['geo_lat'] = pd.to_numeric(df['geo_lat'], errors='coerce')
df['geo_lon'] = pd.to_numeric(df['geo_lon'], errors='coerce')
# df = df.dropna(subset=['geo_lat', 'geo_lon'])

regions_df['area_km2'] = regions_df['area_km2'].astype(str).str.replace(' ', '').astype(float)
regions_df['population'] = regions_df['population'].astype(str).str.replace(' ', '').astype(float)

In [5]:
# Расчет плотности
region_counts = df['region'].value_counts().reset_index()
region_counts.columns = ['region', 'count']
region_density = region_counts.merge(regions_df, on='region', how='left')

region_density['density_per_1000km2'] = region_density['count'] / region_density['area_km2'] * 1000
region_density['density_per_100k'] = (region_density['count'] / region_density['population']) * 100000

In [6]:
# Добавляем рейтинги
max_rank = len(region_density) + 1

region_density['rank_1000km2'] = (
    region_density['density_per_1000km2']
    .rank(ascending=False, method='min')
    .fillna(max_rank)
    .astype(int)
)

region_density['rank_100k'] = (
    region_density['density_per_100k']
    .rank(ascending=False, method='min')
    .fillna(max_rank)
    .astype(int)
)

In [7]:
# Создаем словарь с плотностями и рейтингами
density_dict = region_density.set_index('region')[['density_per_1000km2', 'density_per_100k', 'rank_1000km2', 'rank_100k']].to_dict('index')

In [12]:
# Создание HTML-таблицы регионов с рейтингами
top_regions = region_density.sort_values('density_per_1000km2', ascending=False).head(100)
html = """
<h3 style="margin-bottom: 10px; font-family: Arial, sans-serif; color: #333;">Рейтинги регионов по плотности финансовых организаций</h3>
<table style="width:100%; border-collapse: collapse; font-family: Arial, sans-serif; margin-bottom: 20px;">
    <thead>
        <tr style="background-color: #4CAF50; color: white;">
            <th style="padding: 12px; border: 1px solid #ddd; text-align: left;">Регион</th>
            <th style="padding: 12px; border: 1px solid #ddd; text-align: right;">Кол-во ФО</th>
            <th style="padding: 12px; border: 1px solid #ddd; text-align: right;">ФО на 1000 км²</th>
            <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">Рейтинг</th>
            <th style="padding: 12px; border: 1px solid #ddd; text-align: right;">ФО на 100 тыс.населения</th>
            <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">Рейтинг</th>
        </tr>
    </thead>
    <tbody>
"""
for _, row in top_regions.iterrows():
    # Форматируем числовые значения
    count = f"{int(row['count']):,}".replace(",", " ")  # Формат с пробелами между тысячами
    density_km2 = f"{row['density_per_1000km2']:.2f}" if pd.notna(row['density_per_1000km2']) else 'н/д'
    density_100k = f"{row['density_per_100k']:.2f}" if pd.notna(row['density_per_100k']) else 'н/д'
    
    html += f"""
        <tr style="border-bottom: 1px solid #ddd;">
            <td style="padding: 10px; border: 1px solid #ddd; text-align: left;">{row['region']}</td>
            <td style="padding: 10px; border: 1px solid #ddd; text-align: right;">{count}</td>
            <td style="padding: 10px; border: 1px solid #ddd; text-align: right;">{density_km2}</td>
            <td style="padding: 10px; border: 1px solid #ddd; text-align: center; 
                background-color: {'#ffdddd' if row['rank_1000km2'] == 1 else '#f9f9f9'};
                font-weight: {'bold' if row['rank_1000km2'] <= 3 else 'normal'}">
                {row['rank_1000km2']}
            </td>
            <td style="padding: 10px; border: 1px solid #ddd; text-align: right;">{density_100k}</td>
            <td style="padding: 10px; border: 1px solid #ddd; text-align: center; 
                background-color: {'#ffdddd' if row['rank_100k'] == 1 else '#f9f9f9'};
                font-weight: {'bold' if row['rank_100k'] <= 3 else 'normal'}">
                {row['rank_100k']}
            </td>
        </tr>
    """

html += """
    </tbody>
</table>
<div style="font-size: 12px; color: #666; margin-top: -15px; margin-bottom: 20px;">
    *Рейтинг рассчитан по убыванию плотности (1 - наивысшая плотность)
</div>
"""

In [17]:
# Добавляем заголовок карты
title_html = '''
<h3 align="center" style="font-size:16px; margin-top: 10px; margin-bottom: 10px;">
    <b>Плотность финансовых организаций по регионам РФ</b>
</h3>
'''

In [19]:
# Сохраняем HTML-файл
with open('regions_density_ranking.html', 'w', encoding='utf-8') as f:
    f.write(html)

print("HTML-файл успешно сохранен: regions_density_ranking.html")

HTML-файл успешно сохранен: regions_density_ranking.html


In [23]:
# Подсчет количества уникальных регионов в данных
total_regions = df['region'].nunique()
print(f"Общее количество регионов в данных: {total_regions}")

# Подсчет регионов с данными о площади и населении
regions_with_area = regions_df['region'].nunique()
print(f"Количество регионов с данными о площади: {regions_with_area}")

# Подсчет регионов, для которых можно рассчитать плотность
regions_with_density = region_density[region_density['density_per_1000km2'].notna()]['region'].nunique()
print(f"Количество регионов с рассчитанной плотностью: {regions_with_density}")

# Подсчет регионов без данных (не вошедших в рейтинг)
regions_without_data = total_regions - regions_with_density
print(f"Количество регионов без данных (не вошедших в рейтинг): {regions_without_data}")

# Детальная статистика по регионам
print("\nДетальная статистика:")
print(f"Всего записей организаций: {len(df)}")
print(f"Уникальных регионов в основном датасете: {df['region'].nunique()}")
print(f"Уникальных регионов в справочнике: {regions_df['region'].nunique()}")

# Проверка, какие регионы есть в основном датасете, но отсутствуют в справочнике
missing_in_reference = set(df['region'].unique()) - set(regions_df['region'].unique())
print(f"\nРегионы в основном датасете, но отсутствующие в справочнике: {len(missing_in_reference)}")
if missing_in_reference:
    print("Список отсутствующих регионов:")
    for region in sorted(missing_in_reference):
        print(f"  - {region}")

# Проверка, какие регионы есть в справочнике, но отсутствуют в основном датасете
missing_in_main = set(regions_df['region'].unique()) - set(df['region'].unique())
print(f"\nРегионы в справочнике, но отсутствующие в основном датасете: {len(missing_in_main)}")
if missing_in_main:
    print("Список регионов без организаций:")
    for region in sorted(missing_in_main):
        print(f"  - {region}")

Общее количество регионов в данных: 90
Количество регионов с данными о площади: 89
Количество регионов с рассчитанной плотностью: 89
Количество регионов без данных (не вошедших в рейтинг): 1

Детальная статистика:
Всего записей организаций: 48826
Уникальных регионов в основном датасете: 90
Уникальных регионов в справочнике: 89

Регионы в основном датасете, но отсутствующие в справочнике: 1
Список отсутствующих регионов:
  - г Байконур

Регионы в справочнике, но отсутствующие в основном датасете: 0
